### Optimal parameters model fitting

In [1]:
%load_ext autoreload

In [23]:
import os, sys, importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow.keras as keras

BASE_PATH = os.path.join(os.getcwd(), "..", "..", "..")
MODULE_PATH = os.path.join(BASE_PATH, "modules")
DATASET_PATH = os.path.join(BASE_PATH, "datasets")

sys.path.append(MODULE_PATH)

from bayesian import McDropout
from data import BenchmarkData, DataSetType
from models import setup_growth, FcholletCNN

In [3]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [12]:
mnist = BenchmarkData(DataSetType.MNIST, os.path.join(DATASET_PATH, "mnist"), dtype=np.float32)

In [42]:
val_set_size = 100
test_set_size = 10_000
step_size = 10

In [45]:
x_train, x_test, y_train, y_test = train_test_split(mnist.inputs, mnist.targets, train_size=1000)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_set_size)

In [46]:
x_train.shape

(1000, 28, 28, 1)

In [84]:
learning_rates = np.linspace(0.001, 0.002, 25)
learning_rates

array([0.001     , 0.00104167, 0.00108333, 0.001125  , 0.00116667,
       0.00120833, 0.00125   , 0.00129167, 0.00133333, 0.001375  ,
       0.00141667, 0.00145833, 0.0015    , 0.00154167, 0.00158333,
       0.001625  , 0.00166667, 0.00170833, 0.00175   , 0.00179167,
       0.00183333, 0.001875  , 0.00191667, 0.00195833, 0.002     ])

In [85]:
%autoreload 2
model = FcholletCNN(output=10)
model.build(input_shape=x_train.shape)
model.summary()

Model: "Fchollet-CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1605760   
_________________________________________________________________
dropout_1 (Dropout)          multiple                 

In [86]:
all_histories = []
all_evaluations = []
num_to_train = 3
for i in range(num_to_train):
    
    train_history = []
    evaluation = []
    for lr in learning_rates:
        
        model = FcholletCNN(output=10)
        model.build(input_shape=x_train.shape)
        
        optimizer = keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        
        h = model.fit(x_train, y_train, epochs=20)
        train_history.append(h.history)
        
        evals = model.evaluate(x_test, y_test)
        evaluation.append(evals)
        keras.backend.clear_session()
        
    all_histories.append(train_history)
    all_evaluations.append(evaluation)

Epoch 1/20
32/32 [==============================] - 0s 8ms/step - loss: 10.7935 - accuracy: 0.3950
Epoch 2/20
32/32 [==============================] - 0s 8ms/step - loss: 0.8265 - accuracy: 0.7300
Epoch 3/20
32/32 [==============================] - 0s 8ms/step - loss: 0.5121 - accuracy: 0.8200
Epoch 4/20
32/32 [==============================] - 0s 8ms/step - loss: 0.3753 - accuracy: 0.8710
Epoch 5/20
32/32 [==============================] - 0s 8ms/step - loss: 0.3162 - accuracy: 0.8930
Epoch 6/20
32/32 [==============================] - 0s 8ms/step - loss: 0.2510 - accuracy: 0.9270
Epoch 7/20
32/32 [==============================] - 0s 8ms/step - loss: 0.2294 - accuracy: 0.9310
Epoch 8/20
32/32 [==============================] - 0s 8ms/step - loss: 0.1410 - accuracy: 0.9570
Epoch 9/20
32/32 [==============================] - 0s 8ms/step - loss: 0.1561 - accuracy: 0.9430
Epoch 10/20
32/32 [==============================] - 0s 8ms/step - loss: 0.0943 - accuracy: 0.9650
Epoch 11/20
32/32 

313/313 [==============================] - 2s 6ms/step - loss: 0.3830 - accuracy: 0.9364
Epoch 1/20
32/32 [==============================] - 0s 3ms/step - loss: 8.4255 - accuracy: 0.4220
Epoch 2/20
32/32 [==============================] - 0s 3ms/step - loss: 0.8927 - accuracy: 0.7220
Epoch 3/20
32/32 [==============================] - 0s 4ms/step - loss: 0.5807 - accuracy: 0.8070
Epoch 4/20
32/32 [==============================] - 0s 6ms/step - loss: 0.3711 - accuracy: 0.8650
Epoch 5/20
32/32 [==============================] - 0s 6ms/step - loss: 0.2594 - accuracy: 0.9250
Epoch 6/20
32/32 [==============================] - 0s 7ms/step - loss: 0.2301 - accuracy: 0.9320
Epoch 7/20
32/32 [==============================] - 0s 7ms/step - loss: 0.2083 - accuracy: 0.9260
Epoch 8/20
32/32 [==============================] - 0s 7ms/step - loss: 0.1656 - accuracy: 0.9430
Epoch 9/20
32/32 [==============================] - 0s 7ms/step - loss: 0.1710 - accuracy: 0.9450
Epoch 10/20
32/32 [==========

313/313 [==============================] - 2s 7ms/step - loss: 0.3555 - accuracy: 0.9332
Epoch 1/20
32/32 [==============================] - 0s 7ms/step - loss: 9.1613 - accuracy: 0.3730
Epoch 2/20
32/32 [==============================] - 0s 7ms/step - loss: 0.9220 - accuracy: 0.6970
Epoch 3/20
32/32 [==============================] - 0s 8ms/step - loss: 0.5163 - accuracy: 0.8300
Epoch 4/20
32/32 [==============================] - 0s 8ms/step - loss: 0.4057 - accuracy: 0.8730
Epoch 5/20
32/32 [==============================] - 0s 7ms/step - loss: 0.3189 - accuracy: 0.8960
Epoch 6/20
32/32 [==============================] - 0s 7ms/step - loss: 0.2398 - accuracy: 0.9230
Epoch 7/20
32/32 [==============================] - 0s 7ms/step - loss: 0.1971 - accuracy: 0.9420
Epoch 8/20
32/32 [==============================] - 0s 8ms/step - loss: 0.1623 - accuracy: 0.9440
Epoch 9/20
32/32 [==============================] - 0s 8ms/step - loss: 0.1383 - accuracy: 0.9580
Epoch 10/20
32/32 [==========

32/32 [==============================] - 0s 9ms/step - loss: 0.1104 - accuracy: 0.9700
Epoch 20/20
313/313 [==============================] - 2s 6ms/step - loss: 0.3729 - accuracy: 0.9284
Epoch 1/20
32/32 [==============================] - 0s 7ms/step - loss: 9.8732 - accuracy: 0.4360
Epoch 2/20
32/32 [==============================] - 0s 7ms/step - loss: 0.7306 - accuracy: 0.7580
Epoch 3/20
32/32 [==============================] - 0s 6ms/step - loss: 0.4517 - accuracy: 0.8730
Epoch 4/20
32/32 [==============================] - 0s 7ms/step - loss: 0.2848 - accuracy: 0.9140
Epoch 5/20
32/32 [==============================] - 0s 7ms/step - loss: 0.2102 - accuracy: 0.9300
Epoch 6/20
32/32 [==============================] - 0s 7ms/step - loss: 0.1593 - accuracy: 0.9530
Epoch 7/20
32/32 [==============================] - 0s 7ms/step - loss: 0.1456 - accuracy: 0.9560
Epoch 8/20
32/32 [==============================] - 0s 7ms/step - loss: 0.1275 - accuracy: 0.9570
Epoch 9/20
32/32 [==========

32/32 [==============================] - 0s 7ms/step - loss: 0.1049 - accuracy: 0.9710
Epoch 19/20
32/32 [==============================] - 0s 7ms/step - loss: 0.0887 - accuracy: 0.9680
Epoch 20/20
313/313 [==============================] - 2s 6ms/step - loss: 0.3833 - accuracy: 0.9273
Epoch 1/20
32/32 [==============================] - 0s 9ms/step - loss: 10.5149 - accuracy: 0.4180
Epoch 2/20
32/32 [==============================] - 0s 8ms/step - loss: 0.7763 - accuracy: 0.7380
Epoch 3/20
32/32 [==============================] - 0s 7ms/step - loss: 0.4033 - accuracy: 0.8500
Epoch 4/20
32/32 [==============================] - 0s 8ms/step - loss: 0.3108 - accuracy: 0.8950
Epoch 5/20
32/32 [==============================] - 0s 8ms/step - loss: 0.2406 - accuracy: 0.9120
Epoch 6/20
32/32 [==============================] - 0s 8ms/step - loss: 0.2001 - accuracy: 0.9370
Epoch 7/20
32/32 [==============================] - 0s 7ms/step - loss: 0.1692 - accuracy: 0.9430
Epoch 8/20
32/32 [========

...

32/32 [==============================] - 0s 3ms/step - loss: 0.0692 - accuracy: 0.9760
Epoch 9/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1066 - accuracy: 0.9640
Epoch 10/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0898 - accuracy: 0.9750
Epoch 11/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0876 - accuracy: 0.9730
Epoch 12/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0860 - accuracy: 0.9730
Epoch 13/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0626 - accuracy: 0.9780
Epoch 14/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0560 - accuracy: 0.9840
Epoch 15/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0553 - accuracy: 0.9800
Epoch 16/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0882 - accuracy: 0.9660
Epoch 17/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0535 - accuracy: 0.9830
Epoch 18/20
32/32 [====

32/32 [==============================] - 0s 3ms/step - loss: 0.1446 - accuracy: 0.9550
Epoch 8/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1110 - accuracy: 0.9640
Epoch 9/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0937 - accuracy: 0.9690
Epoch 10/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1133 - accuracy: 0.9640
Epoch 11/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0935 - accuracy: 0.9700
Epoch 12/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0723 - accuracy: 0.9740
Epoch 13/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0815 - accuracy: 0.9740
Epoch 14/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0903 - accuracy: 0.9740
Epoch 15/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1096 - accuracy: 0.9650
Epoch 16/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0712 - accuracy: 0.9740
Epoch 17/20
32/32 [=====

32/32 [==============================] - 0s 3ms/step - loss: 0.2209 - accuracy: 0.9290
Epoch 7/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1422 - accuracy: 0.9510
Epoch 8/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1244 - accuracy: 0.9650
Epoch 9/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1176 - accuracy: 0.9640
Epoch 10/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1065 - accuracy: 0.9620
Epoch 11/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1111 - accuracy: 0.9600
Epoch 12/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1113 - accuracy: 0.9580
Epoch 13/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0874 - accuracy: 0.9730
Epoch 14/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1098 - accuracy: 0.9640
Epoch 15/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0885 - accuracy: 0.9700
Epoch 16/20
32/32 [======

32/32 [==============================] - 0s 3ms/step - loss: 0.2408 - accuracy: 0.9220
Epoch 6/20
32/32 [==============================] - 0s 4ms/step - loss: 0.2222 - accuracy: 0.9210
Epoch 7/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1524 - accuracy: 0.9450
Epoch 8/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1410 - accuracy: 0.9600
Epoch 9/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1422 - accuracy: 0.9480
Epoch 10/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0963 - accuracy: 0.9620
Epoch 11/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1553 - accuracy: 0.9480
Epoch 12/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1154 - accuracy: 0.9620
Epoch 13/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1058 - accuracy: 0.9650
Epoch 14/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1231 - accuracy: 0.9590
Epoch 15/20
32/32 [=======

32/32 [==============================] - 0s 4ms/step - loss: 0.2791 - accuracy: 0.9090
Epoch 5/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1871 - accuracy: 0.9410
Epoch 6/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1644 - accuracy: 0.9450
Epoch 7/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1655 - accuracy: 0.9530
Epoch 8/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1151 - accuracy: 0.9610
Epoch 9/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1310 - accuracy: 0.9560
Epoch 10/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1707 - accuracy: 0.9540
Epoch 11/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1525 - accuracy: 0.9520
Epoch 12/20
32/32 [==============================] - 0s 3ms/step - loss: 0.1402 - accuracy: 0.9500
Epoch 13/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1014 - accuracy: 0.9650
Epoch 14/20
32/32 [========

In [87]:
len(all_evaluations[0])

25

In [88]:
learning_rates

array([0.001     , 0.00104167, 0.00108333, 0.001125  , 0.00116667,
       0.00120833, 0.00125   , 0.00129167, 0.00133333, 0.001375  ,
       0.00141667, 0.00145833, 0.0015    , 0.00154167, 0.00158333,
       0.001625  , 0.00166667, 0.00170833, 0.00175   , 0.00179167,
       0.00183333, 0.001875  , 0.00191667, 0.00195833, 0.002     ])

In [89]:
all_evaluations[1]

[[0.32722344994544983, 0.9348000288009644],
 [0.33598703145980835, 0.9304999709129333],
 [0.3790969252586365, 0.9298999905586243],
 [0.40153810381889343, 0.9276999831199646],
 [0.3624892234802246, 0.9319999814033508],
 [0.37905406951904297, 0.9333000183105469],
 [0.3392460346221924, 0.9366000294685364],
 [0.33076685667037964, 0.9369000196456909],
 [0.3832835555076599, 0.9225000143051147],
 [0.38354188203811646, 0.9301999807357788],
 [0.43043336272239685, 0.9290000200271606],
 [0.3353554904460907, 0.9359999895095825],
 [0.4298847019672394, 0.9243999719619751],
 [0.3711027503013611, 0.9297999739646912],
 [0.37758582830429077, 0.9347000122070312],
 [0.458647757768631, 0.92330002784729],
 [0.37079668045043945, 0.9351000189781189],
 [0.3794003129005432, 0.9320999979972839],
 [0.3782902956008911, 0.9300000071525574],
 [0.3804357647895813, 0.9225999712944031],
 [0.4346923232078552, 0.9290000200271606],
 [0.40345096588134766, 0.9262999892234802],
 [0.3457295596599579, 0.9301999807357788],
 [0.